In [1]:
%%writefile chatbot-streamlit.py

import oracledb
import oci
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain_community.embeddings import OCIGenAIEmbeddings
from LoadProperties import LoadProperties

print("Successfully imported libraries and modules")

# Setup basic variables
properties = LoadProperties()

# Declare username and password and dsn (data connection string)
un = "vector"
pw = "vector"
cs = "localhost/FREEPDB1"

# Connect to the database
try:
    conn23c = oracledb.connect(user=un, password=pw, dsn=cs)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!", e)

# Step 1 - create the chain
def create_chain():
    embed_model = OCIGenAIEmbeddings(
        model_id=properties.getEmbeddingModelName(),
        service_endpoint=properties.getEndpoint(),
        compartment_id=properties.getCompartment(),
        auth_type="INSTANCE_PRINCIPAL",
    )

    vs = OracleVS(
        embedding_function=embed_model,
        client=conn23c,
        table_name="DEMO_TABLE",
        distance_strategy=DistanceStrategy.DOT_PRODUCT,
    )

    retv = vs.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    llm = ChatOCIGenAI(
        model_id=properties.getModelName(),
        service_endpoint=properties.getEndpoint(),
        compartment_id=properties.getCompartment(),
        auth_type="INSTANCE_PRINCIPAL",
        model_kwargs={"max_tokens": 200},
    )

    memory = ConversationBufferMemory(
        llm=llm,
        memory_key="chat_history",
        return_messages=True,
        output_key="answer",
    )

    qa = ConversationalRetrievalChain.from_llm(
        llm, retriever=retv, memory=memory, return_source_documents=True
    )
    return qa

# Step 2 - define chat function
def chat(llm_chain, user_input):
    # generate a prediction for a prompt
    bot_json = llm_chain.invoke(user_input)
    return {"bot_response": bot_json}


# Step 3 - Streamlit app interface
if __name__ == "__main__":
    import streamlit as st

    st.subheader("Chatbot that answers your study questions")
    col1, col2 = st.columns([4, 1])

    def initialize_session_state():
        if "llm_chain" not in st.session_state:
            st.session_state["llm_chain"] = create_chain()
        return st.session_state["llm_chain"]

    user_input = st.chat_input()

    with col1:
        col1.subheader("------Ask me a question about science chapters------")
        # col2.subheader("References")

        if "messages" not in st.session_state:
            st.session_state.messages = []

        if user_input:
            llm_chain = initialize_session_state()
            bot_response = chat(llm_chain, user_input)
            print("bot_response->\n", bot_response)
            st.session_state.messages.append({
                "role": "chatbot",
                "content": bot_response
            })

        for message in st.session_state.messages:
            st.chat_message("user").write(
                "Question: " + str(message["content"]["bot_response"].get("question", "[No Question Found]"))
            )

            st.chat_message("assistant").write(
                "Answer: " + str(message["content"]["bot_response"].get("answer", "[No Answer Found]"))
            )

            for doc in message["content"]["bot_response"].get("source_documents", []):
                st.chat_message("assistant").write(
                    "Chunk Id Is: " + str(doc.metadata["id"]) +
                    "\n-page->" + str(doc.metadata["link"])
                )


Overwriting chatbot-streamlit.py


In [2]:
!streamlit run chatbot-streamlit.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.0.0.73:8501
  External URL: http://89.168.86.230:8501

Successfully imported libraries and modules
Connection successful!
Successfully imported libraries and modules
Connection successful!
bot_response->
 {'bot_response': {'question': 'how many modules are there in the AI foundation course?', 'chat_history': [HumanMessage(content='how many modules are there in the AI foundation course?'), AIMessage(content='There are six modules in the AI Foundations course.')], 'answer': 'There are six modules in the AI Foundations course.', 'source_documents': [Document(metadata={'id': '0', 'link': 'Page 0'}, page_content="Course Name: Oracle Cloud Infrastructure AI Foundations Module 1: Welcome to AI Foundations  1.1 Welcome to AI Foundations  [AUDIO LOGO] Hello, and welcome to the course introduction to the OCI AI Foundations 2023 course. I'm Hemant Gahankari. I'm a senior principa